In [1]:
import numpy as np
import pandas as pd

import ast
import random
import lmstudio as lms

from IPython.display import Markdown

In [2]:
raw_link = pd.read_csv('../../Data/sampling_gpt.csv')
raw_tthc = pd.read_csv('../../Data/postprocessed_tthc.csv', dtype={'maThuTuc': str})

In [3]:
raw_link['TTHCLienQuan'] = raw_link['TTHCLienQuan'].apply(ast.literal_eval)
pd.set_option('future.no_silent_downcasting', True)
def getKnowledge(s):
     l = raw_tthc[raw_tthc['link'].isin(s)].replace("Không có thông tin", 
                                                    np.nan).drop(columns=['link']).reset_index(drop=True).dropna(axis=1, how='all')
     return "\n" + l.to_string() + "\n"

In [4]:
template_df = pd.read_csv('../../Template/template.csv', index_col=0)
config_df = pd.read_csv('../../Template/config.csv', index_col=0)

In [5]:
cols = ['link', 'Mã thủ tục hành chính liên quan', 'Số quyết định', 'Tên thủ tục', 'Cấp thực hiện', 'Loại thủ tục', 'Lĩnh vực', 
        'Trình tự thực hiện', 'Cách thức thực hiện', 'Thành phần hồ sơ', 'Đối tượng thực hiện', 'Cơ quan thực hiện', 
        'Cơ quan có thẩm quyền', 'Địa chỉ tiếp nhận hồ sơ', 'Cơ quan được ủy quyền', 'Cơ quan phối hợp', 'Kết quả thực hiện', 
        'Căn cứ pháp lý', 'Yêu cầu, điều kiện thực hiện', 'Từ khóa', 'Mô tả']

In [6]:
raw_tthc.columns = cols

In [7]:
pattern = []

pattern.append("Bạn đang cố gắng xác định xem câu trả lời có hiểu sai ngữ cảnh và mục đích của câu hỏi hay không.")
pattern.append("Bạn đang cố gắng xác định xem câu trả lời có sự mâu thuẫn về mặt thực tế so với kiến thức liên quan hay không. Một số thông tin trong câu trả lời có thể đã được bịa ra.")
pattern.append("Bạn đang cố gắng xác định xem câu trả lời có quá chung chung hoặc quá chi tiết so với mức độ cụ thể phù hợp để trả lời câu hỏi hay không.")
pattern.append("Bạn đang cố gắng xác định xem câu trả lời có thể được suy ra đúng từ kiến thức liên quan hay không.")

In [8]:
evaluate_context = template_df['evaluate_context']['open_source']
print(evaluate_context)

Bạn sẽ đóng vai trò là một người đánh giá câu trả lời (answer judge). Với một câu hỏi, câu trả lời, và kiến thức liên quan, mục tiêu của bạn là xác định xem câu trả lời được cung cấp có chứa thông tin không đúng sự thật hoặc thông tin ảo giác (hallucinated information) hay không.
{pattern}
Bạn nên cố gắng hết sức để xác định xem câu trả lời có chứa thông tin không đúng sự thật hoặc thông tin ảo giác hay không. Câu trả lời bạn đưa ra bắt buộc CHỈ là "Có" hoặc "Không", và không giải thích gì thêm. Trả lời "Có" nếu câu trả lời chứa thông tin ảo giác, trả lời "Không" nếu câu trả lời không chứa thông tin ảo giác.



In [9]:
evaluate_template = template_df['evaluate_template']['open_source']
print(evaluate_template)

#Kiến thức liên quan#: {knowledge}

#Câu hỏi#: {question}

#Câu trả lời#: {answer}

#Đánh giá của bạn#:



In [10]:
config = {"temperature": float(config_df['temperature']['open_source']), 
          "maxTokens": float(config_df['max_tokens']['open_source']), 
          "topPSampling": float(config_df['top_p']['open_source'])
          }
config

{'temperature': 0.7, 'maxTokens': 32.0, 'topPSampling': 0.9}

In [11]:
model_name = "wizardlm-2-7b"

model = lms.llm(model_name)

filename = f'{model_name}_evaluate.csv'

cols = ['link', 'cauTraLoiDung', 'danhGiaDung', 'cauTraLoiAoGiac', 'pattern', 'danhGiaAoGiac']

try:
    df = pd.read_csv(filename)
    if df.empty:
        print("CSV file is empty")
        df = pd.DataFrame(columns=cols)
except FileNotFoundError:
    print(f"File not found: {filename}")
    df = pd.DataFrame(columns=cols)
    df.to_csv(filename, index=False)
except pd.errors.EmptyDataError:
    print(f"File is completely empty: {filename}")
    df = pd.DataFrame(columns=cols)
    df.to_csv(filename, index=False)

for i in range(len(raw_link)):
    if raw_link['link'][i] in list(df['link'].values):
        tmp = df[df['link']==raw_link['link'][i]].iloc[0]
        if (not pd.isna(tmp['danhGiaDung'])) and (not pd.isna(tmp['danhGiaAoGiac'])):
            continue

    p = raw_link['pattern'][i]
    question = raw_link['cauHoi'][i]
    right_answer = raw_link['cauTraLoi'][i]
    hallucinated_answer = raw_link['cauTraLoiAoGiac'][i]

    context_right = evaluate_context.format(pattern="")
    context_hallucinated = evaluate_context.format(pattern=pattern[p])

    knowledge = getKnowledge(raw_link['TTHCLienQuan'][i])
    prompt_right = evaluate_template.format(knowledge=knowledge, question=question, answer=hallucinated_answer)
    prompt_hallucinated = evaluate_template.format(knowledge=knowledge, question=question, answer=hallucinated_answer)

    try:
        completion = model.respond(history={"messages": [
            {"role": "system", "content": context_right},
            {"role": "user", "content": prompt_right}
        ]}, config=config)
        output_right = str(completion)
        completion = model.respond(history={"messages": [
            {"role": "system", "content": context_hallucinated},
            {"role": "user", "content": prompt_hallucinated}
        ]}, config=config)
        output_hallucinated = str(completion)
    except Exception as e:
        output_right = ""
        output_hallucinated = ""
        # print(e)
        # break

    results = [raw_link['link'][i], right_answer, output_right, hallucinated_answer, p, output_hallucinated]
    df.loc[i, cols] = results    
    df.to_csv(filename, index=False)

    print(f'Done of {len(raw_link)}: {i+1}', end='\r')
    

In [12]:
df.info()
df.tail()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 6 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   link             1000 non-null   object
 1   cauTraLoiDung    1000 non-null   object
 2   danhGiaDung      1000 non-null   object
 3   cauTraLoiAoGiac  1000 non-null   object
 4   pattern          1000 non-null   int64 
 5   danhGiaAoGiac    1000 non-null   object
dtypes: int64(1), object(5)
memory usage: 47.0+ KB


,link,cauTraLoiDung,danhGiaDung,cauTraLoiAoGiac,pattern,danhGiaAoGiac
995,https://dichvucong.gov.vn/p/home/dvc-chi-tiet-...,03 (ba) tháng trước thời điểm Giấy chứng nhận ...,Không - Thông tin cung cấp trong câu hỏi chứa ...,Để cấp lại Giấy chứng nhận Vùng an toàn dịch b...,3,Câu trả lời không chứa thông tin ảo giác. Có một
996,https://dichvucong.gov.vn/p/home/dvc-chi-tiet-...,Trả lời: Theo quy định tại Thông tư liên tịch ...,Có,Trả lời: Theo quy định tại Nghị định số 159/20...,3,Câu trả lời đã cung cấp một số thông tin không đ
997,https://dichvucong.gov.vn/p/home/dvc-chi-tiet-...,"Theo điểm a, b Khoản 5 Điều 18, Thông tư 66/20...",\nThảm thức Hình câm thiếu nóc nhấn có thể đư,"Theo quy định tại Điều 22, Mục 3 của Thông tư ...",3,\nThêm tiều ...
998,https://dichvucong.gov.vn/p/home/dvc-chi-tiet-...,"Điều 14, Khoản 2 Điều 15, Điều 16 và Điều 17 N...",b c b\n d a d m ( b t k in as\n m n v the v c...,"Để được cấp Giấy phép hoạt động dự báo, cảnh b...",3,k g B the b d m v\n m b má d a t k b c b t k...
999,https://dichvucong.gov.vn/p/home/dvc-chi-tiet-...,Tại Điểm b Khoản 1 Điều 13 Nghị định số 60/201...,\n\n\n ---------------- ...,"Để được cấp lại giấy phép hoạt động in, công t...",3,\n\n\n 9 v\n\n\n\n\n\n*********...
